# MNIST Classification
> Performance analysis for MNIST Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [l3, performance, MNIST]
- image: images/mnist.png

# Theoretical Analysis of MNIST

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [1]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'mnist')

alt.LayerChart(...)

### Performance Prediction

The plot below shows ... Lorem Ipsum ...

In [2]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load mnist dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_mnist.csv"), 'red', 'Performance Prediction for MNIST')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST 

In [16]:
#hide_input
%run scripts/altair_plots.py  #get table with the experiments overview
#dataframe = pd.read_csv('data/overview_experiments_mnist.csv')
display(pd.read_csv('data/overview_experiments_mnist.csv'))

,Unnamed: 0,Unnamed: 1,MNIST Classification,Unnamed: 3
0,Hardware,Platform,MLP,Batch/Stream/Thread
1,FPGA,ZCU102-DPU,na,"[1,2,3,4,5,6,7,8]"
2,FPGA,ZCU104-DPU,na,"[1,2,3,4,5,6,7,8]"
3,FPGA,Ultra96-DPU,na,"[1,2,3,4,5,6,7,8]"
4,FPGA,ZCU104-FINN,"[INT2, INT4] * [100%,50%,25%,12.5%]","[1,2,4,8,16,32,64,128,256,512,10000]"
5,FPGA,ZCU104-BISMO,"[INT2, INT4] * [100%,50%,25%,12.5%]","[2,4,8,16,32,64,128]"
6,GPU,TX2-maxn,"[FP16, FP32] * [100%,50%,25%,12.5%]","[1,2,4,8,16,32,64,128]"
7,GPU,TX2-maxp,"[FP16, FP32] * [100%,50%,25%,12.5%]","[1,2,4,8,16,32,64,128]"
8,GPU,TX2-maxq,"[FP16, FP32] * [100%,50%,25%,12.5%]","[1,2,4,8,16,32,64,128]"
9,TPU,TPU-fast clk,na,[1]


In [5]:
import pandas as pd
pd.set_option('max_colwidth',70)
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({'A': ['bdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbbdbdbdbdbdbdbdbd', 'foo', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})
df

,A,B
0,bdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbdbbdbdbdbdbdbdbdbd,abc
1,foo,bar
2,bait,xyz


In [4]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

csv_path = "./data/cleaned_csv/backup.csv"

In [5]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_quant_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
mlp_df = maxp_df[(maxp_df["NN_Topology"] == "MLP") & maxp_df['lat-comp'].notna()]
mlp_df["hw_quant_prun"] = mlp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
mlp_df["PruningFactor"] = mlp_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(mlp_df, "lat-comp", "NN_Topology");
mlp_df["quant_model"] = mlp_df.Precision + '_' + mlp_df.HWType

mnist_df    = maxp_df[(maxp_df.NN_Topology == 'MLP') & maxp_df["top1 [%]"].notna()]
mnist_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [7]:
#hide
figa_df = mlp_df[(mlp_df["HWType"].isin(["NCS", "ZCU104-Bismo", "A53-gemmlowp"]))]
figb_df = mlp_df[(mlp_df["HWType"].isin(["GPU", "ZCU104-FINN", "A53-gemmlowp"]))]

### Line Plot

In [8]:
#hide_input
fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_quant_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized MLP Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [9]:
#hide_input
box1 = alt.Chart(mlp_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for MLP"
).interactive()
box1

alt.FacetChart(...)

In [10]:
#hide_input
box1 = alt.Chart(mlp_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for MLP",
).interactive()
box1

alt.FacetChart(...)

In [ ]:
#hide_input
box1 = alt.Chart(mlp_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for MLP",
).interactive()
box1

### Pareto Graphs

In [11]:
#hide_input
mnist_pareto = get_pareto_df(mnist_df, 'hw_quant_prun', 'fps-comp', 'top1')

mnist_lines = alt.Chart(mnist_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_quant_prun", legend=alt.Legend(columns=1)),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
mnist_pareto_plot = alt.Chart(mnist_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(mnist_lines+mnist_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="MNIST Cassification Design Space: Accuracy versus Performance"
)

alt.LayerChart(...)

In [12]:
# Measurements

In [21]:
#hide
mnist_df.to_csv('data/processed_csv/experimental_data_mnist.csv', index = False)
mnist_df.head(30)

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],BasePWR [W],IdlePWR [W],FullPwr [W],GOPS,PruningFactor,level,hw_quant_prun
131,MLP,NCS,FP16,na,1,2.892570,218.893,345.713,1.84518,2.91422,98.86,NaN,0.53,1.20,1.7940,NaN,100%,l3,NCS_FP16_100%
132,MLP,NCS,FP16,na,2,4.684590,247.308,426.931,2.08471,3.59886,98.86,NaN,0.53,1.20,1.9490,NaN,100%,l3,NCS_FP16_100%
133,MLP,NCS,FP16,na,4,8.118210,269.491,492.719,2.27170,4.15343,98.86,NaN,0.53,1.20,1.9460,NaN,100%,l3,NCS_FP16_100%
134,MLP,NCS,FP16,na,8,14.803800,282.686,540.400,2.38293,4.55536,98.86,NaN,0.53,1.20,2.0840,NaN,100%,l3,NCS_FP16_100%
135,MLP,NCS,FP16,na,16,27.493500,292.848,581.956,2.46859,4.90565,98.86,NaN,0.53,1.20,2.1360,NaN,100%,l3,NCS_FP16_100%
136,MLP,NCS,FP16,na,32,53.057200,297.970,603.122,2.51177,5.08408,98.86,NaN,0.53,1.20,2.1350,NaN,100%,l3,NCS_FP16_100%
137,MLP,NCS,FP16,na,64,104.435000,301.514,612.823,2.54164,5.16586,98.86,NaN,0.53,1.20,2.1860,NaN,100%,l3,NCS_FP16_100%
138,MLP,NCS,FP16,na,128,206.007000,303.434,621.337,2.55783,5.23762,98.86,NaN,0.53,1.20,2.2120,NaN,100%,l3,NCS_FP16_100%
157,MLP,GPU,FP16,maxp,1,0.698266,976.168,1466.410,8.22870,12.36130,97.30,NaN,1.80,4.70,9.8810,NaN,100%,l3,GPU_FP16_100%
158,MLP,GPU,FP16,maxp,2,0.739980,1635.780,2785.550,13.78900,23.48100,97.30,NaN,1.80,4.70,9.9030,NaN,100%,l3,GPU_FP16_100%
